In [1]:
#!create a data directory
%mkdir data

mkdir: data: File exists


In [2]:
#Import Dependencies
import pandas as pd
from sodapy import Socrata

In [3]:
#Import user credentials
from config import *

In [4]:
client = Socrata('data.lacity.org', App_Token, username=username,password=password)

In [5]:
results = client.get("k8cc-2d49", limit=500000)

In [6]:
#Store the data in a csv format for reusability
results_df = pd.DataFrame.from_records(results)

In [8]:
results_df.to_csv("data/collision.csv", index=False)

In [37]:
#Read the stored collision data
collisionDF = pd.read_csv('data/collision.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
#Print Sample data
collisionDF[:5]

,area,area_name,crm_cd,crm_cd_desc,cross_street,date_occ,date_rptd,dr_no,location,location_1,mocodes,premis_cd,premis_desc,rpt_dist_no,time_occ,vict_age,vict_descent,vict_sex
0,1,Central,997,TRAFFIC COLLISION,2ND,2010-04-20T00:00:00.000,NaN,100109637,ALAMEDA,"{'type': 'Point', 'coordinates': [-118.2381, 3...",NaN,101.0,STREET,138,1235,55.0,B,M
1,7,Wilshire,997,TRAFFIC COLLISION,RIMPAU,2010-12-21T00:00:00.000,NaN,100719446,PICO,"{'type': 'Point', 'coordinates': [-118.337, 34...",NaN,101.0,STREET,758,2218,37.0,B,M
2,1,Central,997,TRAFFIC COLLISION,BROADWAY,2010-06-04T00:00:00.000,NaN,100111904,5TH,"{'type': 'Point', 'coordinates': [-118.2507, 3...",NaN,101.0,STREET,153,2310,29.0,H,M
3,1,Central,997,TRAFFIC COLLISION,SAN PEDRO,2010-03-22T00:00:00.000,NaN,100108154,4TH,"{'type': 'Point', 'coordinates': [-118.2422, 3...",NaN,101.0,STREET,148,1325,43.0,B,F
4,13,Newton,997,TRAFFIC COLLISION,ADAMS,2010-01-12T00:00:00.000,NaN,101304785,HILL,"{'type': 'Point', 'coordinates': [-118.2708, 3...",NaN,101.0,STREET,1321,1800,34.0,O,M


In [39]:
#Only needed columns are save in the new Collision DataFrame
collisionDF= pd.DataFrame({'Age': collisionDF.vict_age,
                          'Descent': collisionDF.vict_descent, 
                          'Sex':collisionDF.vict_sex,
                          'Area':collisionDF.area_name, 
                          'Location':collisionDF.location, 
                          'Intersection': collisionDF.cross_street,
                          'GeoJson':collisionDF.location_1, 
                          'Date':collisionDF.date_occ, 
                          'Time':collisionDF.time_occ,
                          'Reporting District':collisionDF.rpt_dist_no
                         })

In [40]:
#Check the count ofevery columns
collisionDF.count()

Age                   382396
Area                  418509
Date                  418509
Descent               412457
GeoJson               418509
Intersection          400211
Location              418509
Reporting District    418509
Sex                   412902
Time                  418509
dtype: int64

In [41]:
#Reorganize the columns so that its more readable
collisionDF = collisionDF[['Age', 'Descent', 'Sex','Area', 'Location', 'Intersection',
                           'GeoJson', 'Date', 'Time','Reporting District']]

In [42]:
#Format the date in the Date column
collisionDF.Date = pd.to_datetime(collisionDF.Date)
collisionDF[:10]

,Age,Descent,Sex,Area,Location,Intersection,GeoJson,Date,Time,Reporting District
0,55.0,B,M,Central,ALAMEDA,2ND,"{'type': 'Point', 'coordinates': [-118.2381, 3...",2010-04-20,1235,138
1,37.0,B,M,Wilshire,PICO,RIMPAU,"{'type': 'Point', 'coordinates': [-118.337, 34...",2010-12-21,2218,758
2,29.0,H,M,Central,5TH,BROADWAY,"{'type': 'Point', 'coordinates': [-118.2507, 3...",2010-06-04,2310,153
3,43.0,B,F,Central,4TH,SAN PEDRO,"{'type': 'Point', 'coordinates': [-118.2422, 3...",2010-03-22,1325,148
4,34.0,O,M,Newton,HILL,ADAMS,"{'type': 'Point', 'coordinates': [-118.2708, 3...",2010-01-12,1800,1321
5,77.0,B,F,West LA,PICO,CARDIFF,"{'type': 'Point', 'coordinates': [-118.3918, 3...",2010-09-20,1635,857
6,74.0,H,F,Harbor,WEYMOUTH,9TH,"{'type': 'Point', 'coordinates': [-118.3097, 3...",2010-11-19,1529,561
7,22.0,H,M,Mission,ORION,CABRITO,"{'type': 'Point', 'coordinates': [-118.4721, 3...",2010-07-25,800,1991
8,54.0,W,M,Hollywood,LAUREL CANYON BL,MULHOLLAND DR,"{'type': 'Point', 'coordinates': [-118.3753, 3...",2010-10-14,1030,621
9,NaN,H,M,Olympic,OLYMPIC BL,WESTERN AV,"{'type': 'Point', 'coordinates': [-118.3091, 3...",2010-08-31,1000,2053


In [43]:
#Drop records entries for columns with NaN values
cleanDF = collisionDF.dropna(subset = ['Age', 'Descent', 'Sex'])

In [44]:
#Print a sample data
cleanDF[:5]

,Age,Descent,Sex,Area,Location,Intersection,GeoJson,Date,Time,Reporting District
0,55.0,B,M,Central,ALAMEDA,2ND,"{'type': 'Point', 'coordinates': [-118.2381, 3...",2010-04-20,1235,138
1,37.0,B,M,Wilshire,PICO,RIMPAU,"{'type': 'Point', 'coordinates': [-118.337, 34...",2010-12-21,2218,758
2,29.0,H,M,Central,5TH,BROADWAY,"{'type': 'Point', 'coordinates': [-118.2507, 3...",2010-06-04,2310,153
3,43.0,B,F,Central,4TH,SAN PEDRO,"{'type': 'Point', 'coordinates': [-118.2422, 3...",2010-03-22,1325,148
4,34.0,O,M,Newton,HILL,ADAMS,"{'type': 'Point', 'coordinates': [-118.2708, 3...",2010-01-12,1800,1321


In [45]:
#Check the count of records per column
cleanDF.count()

Age                   379407
Descent               379407
Sex                   379407
Area                  379407
Location              379407
Intersection          364867
GeoJson               379407
Date                  379407
Time                  379407
Reporting District    379407
dtype: int64

In [46]:
#Save the cleaned collision data to a csv file
cleanDF.to_csv("data/Cleaned_collision.csv", index=False)